In [1]:
!pip install transformers datasets rouge_score nltk evaluate

In [2]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from datasets import load_dataset
import evaluate
from nltk.translate.bleu_score import sentence_bleu

In [3]:
# Load Multi-News test dataset (1/10th of the dataset)
dataset = load_dataset("multi_news", split="test")
subset_size = len(dataset) // 20  # Take only 20% of the dataset

# Extract a subset of test samples and truncate text length to prevent CUDA errors
test_samples = [doc[:1024] for doc in dataset["document"][:subset_size]]  # Cut articles to 1024 chars max
reference_summaries = dataset["summary"][:subset_size]

print(f"Using {subset_size} test samples for evaluation.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Using 281 test samples for evaluation.


In [4]:
# Define model name
model_name = "PalosGW/distilbart-finetuned-multinews-M"

# Load model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f"Model loaded on: {device}")

/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


Model loaded on: cpu


In [5]:
# Tokenize inputs with truncation
inputs = tokenizer(test_samples, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")

# Move inputs to GPU if available
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate summaries
outputs = model.generate(**inputs, max_length=512, num_return_sequences=1)
generated_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print a sample
print("\n🔹 Example Input Document:")
print(test_samples[0])
print("\n🔹 Reference Summary:")
print(reference_summaries[0])
print("\n🔹 Model-Generated Summary:")
print(generated_summaries[0])

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1528: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(



🔹 Example Input Document:
GOP Eyes Gains As Voters In 11 States Pick Governors 
 
 Enlarge this image toggle caption Jim Cole/AP Jim Cole/AP 
 
 Voters in 11 states will pick their governors tonight, and Republicans appear on track to increase their numbers by at least one, with the potential to extend their hold to more than two-thirds of the nation's top state offices. 
 
 Eight of the gubernatorial seats up for grabs are now held by Democrats; three are in Republican hands. Republicans currently hold 29 governorships, Democrats have 20, and Rhode Island's Gov. Lincoln Chafee is an Independent. 
 
 Polls and race analysts suggest that only three of tonight's contests are considered competitive, all in states where incumbent Democratic governors aren't running again: Montana, New Hampshire and Washington. 
 
 While those state races remain too close to call, Republicans are expected to wrest the North Carolina governorship from Democratic control, and to easily win GOP-held seats in 

In [6]:
rouge = evaluate.load("rouge")
rouge_scores = rouge.compute(predictions=generated_summaries, references=reference_summaries)
print("\n🔹 ROUGE Scores:", rouge_scores)


🔹 ROUGE Scores: {'rouge1': 0.08551231350951005, 'rouge2': 0.0293136075897647, 'rougeL': 0.04612877374883992, 'rougeLsum': 0.046324919096646155}
